In [ ]:
import preamble
from virtual_dynamics import SimpleVirtualDynamics
from robot import Robot
from app_loop import AppLoop
from vocalizer import Vocalizer
from plot_client import PlotClient
import numpy as np
from numpy.typing import NDArray
import pandas as pd

In [ ]:
class RobotApp(AppLoop):
    MASS = 10.0
    DAMPENING = 20.0
    AXES = Robot.TRANSLATION

    def __init__(self, use_plotter: bool = False):
        super().__init__()
        self.use_plotter = use_plotter

    def startup(self) -> None:
        self.robot = Robot("169.254.9.43",
                           translational_force_deadband=6.0,
                           rotational_force_deadband=0.5)
        self.robot.control.zeroFtSensor()
        self.p0 = self.robot.INIT_POSE[:3]

        self.dynamics = SimpleVirtualDynamics(M=self.MASS, B=self.DAMPENING, K=0)

        self.path, self.v_path = self.load_path('../data/reference_path.csv')
        self.forward_state = True

        self.vocalizer = Vocalizer(buffer_period=4.0)

        if self.use_plotter:
            self.plotter = PlotClient()
            self.plotter.create_plot("F_h", title="$F_h(t)$", xlabel="Time (s)", ylabel="Force (N)")
            self.plotter.create_line("F_h", "x", plot_style='-', label='x', color='red')
            self.plotter.create_line("F_h", "y", plot_style='-', label='y', color='green')
            self.plotter.create_line("F_h", "z", plot_style='-', label='z', color='blue')

    def generate_path(self, start_point: NDArray, end_point: NDArray, N: int) -> NDArray:
        length = np.linalg.norm(end_point - start_point)
        direction = (end_point - start_point) / length
        steps = np.linspace(0.0, length, N)
        return np.array([start_point + direction * step for step in steps] + [end_point - direction * step for step in steps])

    def load_path(self, file_path: str) -> NDArray:
        data = pd.read_csv(file_path)

        path = np.stack([data['p_x'].values, data['p_y'].values, data['p_z'].values], axis=1)
        path += self.p0 - path[0]

        v_path = np.stack([data['v_x'].values, data['v_y'].values, data['v_z'].values], axis=1)

        return path, v_path

    def get_target(self, p: NDArray) -> NDArray:
        distances = np.linalg.norm(self.path - p, axis=1)
        index = np.argmin(distances)
        return self.path[index], self.v_path[index]

    def update(self, t: float, dt: float) -> None:
        period_start = self.robot.control.initPeriod()

        p = self.robot.get_pose(self.AXES)
        if p[0] <= self.path[-1, 0] if self.forward_state else p[0] >= self.path[0, 0]:
            self.forward_state = not self.forward_state
            self.v_path *= -1.0

        F_ext = self.robot.get_force(self.AXES)
        self.dynamics.apply_force(F_ext, dt)
        self.robot.set_velocity(self.dynamics.get_velocity(),
                                Robot.TRANSLATION, acceleration=10)

        self.robot.control.waitPeriod(period_start)

        if self.use_plotter:
            self.plotter.update_line("F_h", "x", (t, F_ext[0]), mode='append')
            self.plotter.update_line("F_h", "y", (t, F_ext[1]), mode='append')
            self.plotter.update_line("F_h", "z", (t, F_ext[2]), mode='append')
            self.plotter.config_plot("F_h", ylim=(-40, 40), xlim=(t - 10.0, t))

    def shutdown(self) -> None:
        self.robot.set_velocity(Robot.zeroed_translation_rotation())
        if self.use_plotter:
            self.plotter.close()

In [ ]:
app = RobotApp(use_plotter=True)
app.run()